<a href="https://colab.research.google.com/github/Zhenriquee/HEALTH_MARKET_VISION/blob/main/Extra%C3%A7%C3%A3o_e_Tratamento_dos_Dados_ANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coleta e Tratamento de Dados da ANS

## Objetivo:
Realizar a extração dos dados da ANS com a finalidade de analisar o posicionamento da Unimed Caruaru com Relação as outras Operadoras, no final iremos armazenar esses dados em um arquivo .db e Utilizar o streamlit para projeção desses dados.

## Links Utilizados para Extração


*   [Qtd. Beneficiarios por Trimestre](https://dadosabertos.ans.gov.br/FTP/Base_de_dados/Microdados/dados_dbc/beneficiarios/operadoras/)
*   [Demonstração Contabeis](https://dadosabertos.ans.gov.br/FTP/PDA/demonstracoes_contabeis/)
*   [Dimensão Operadora](https://dados-abertos-service.pr.ans.gov.br/swagger-ui/index.html?configUrl=/v3/api-docs/swagger-config#/)


### Bibliotecas Utilizadas

In [2]:
pip install datasus-dbc dbfread requests curl_cffi bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00


In [3]:
import os
import requests
import sqlite3
import pandas as pd
import uuid
import re
import io
import zipfile
import unicodedata
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datasus_dbc import decompress
from dbfread import DBF
from concurrent.futures import ProcessPoolExecutor, as_completed
from curl_cffi import requests as requests_curl
from concurrent.futures import ThreadPoolExecutor, as_completed

## Inicio Extração e Tratamento Qtd. Beneficiarios por Trimestre



In [4]:
# --- FUNÇÕES AUXILIARES (Worker) ---
# Precisam estar fora da classe para o multiprocessing funcionar bem no Windows

def _gerar_chave_trimestre(id_cmpt):
    try:
        s_cmpt = str(id_cmpt).strip()
        if len(s_cmpt) < 6: return None
        ano = s_cmpt[:4]
        mes = int(s_cmpt[4:6])
        trimestre = (mes - 1) // 3 + 1
        return f"{ano}-T{trimestre}"
    except:
        return None

def processar_arquivo_worker(link):
    """
    Função isolada que roda em um núcleo separado da CPU.
    Baixa, Converte, Filtra e Agrupa. Retorna um DataFrame pronto (ou None).
    """
    nome_arquivo = link.split('/')[-1]

    # Gera nomes únicos para evitar colisão entre processos
    id_unico = str(uuid.uuid4())
    temp_dbc = f"temp_{id_unico}.dbc"
    temp_dbf = f"temp_{id_unico}.dbf"

    colunas_desejadas = ['ID_CMPT', 'CD_OPERADO', 'NR_BENEF_T']
    resultado_df = None

    try:
        # 1. Download
        r = requests.get(link, stream=True, timeout=30)
        with open(temp_dbc, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

        # 2. Descompressão
        decompress(temp_dbc, temp_dbf)

        # 3. Leitura e Pandas
        table = DBF(temp_dbf, encoding='iso-8859-1', load=True)
        df = pd.DataFrame(iter(table))

        if not df.empty:
            # Verifica colunas
            if all(col in df.columns for col in colunas_desejadas):
                df = df[colunas_desejadas].copy()
                df['NR_BENEF_T'] = pd.to_numeric(df['NR_BENEF_T'], errors='coerce').fillna(0)

                # Agrupa e Soma (Reduzindo drasticamente o tamanho dos dados antes de retornar)
                df_agrupado = df.groupby(['ID_CMPT', 'CD_OPERADO'], as_index=False)['NR_BENEF_T'].sum()

                # Cria chave Trimestre
                df_agrupado['ID_TRIMESTRE'] = df_agrupado['ID_CMPT'].apply(_gerar_chave_trimestre)

                resultado_df = df_agrupado
            else:
                print(f"   [Worker] Ignorado {nome_arquivo}: Colunas ausentes.")

    except Exception as e:
        print(f"   [Worker] Erro em {nome_arquivo}: {e}")

    finally:
        # Limpeza rigorosa dos arquivos temporários deste processo
        if os.path.exists(temp_dbc): os.remove(temp_dbc)
        if os.path.exists(temp_dbf): os.remove(temp_dbf)

    return resultado_df

# --- CLASSE PRINCIPAL ---

class ImportadorANSParalelo:
    def __init__(self, db_path='dados_ans.db'):
        self.db_path = db_path

    def etapa_1_e_2_obter_links(self, url_origem):
        print(f"--- Mapeando arquivos em: {url_origem} ---")
        try:
            response = requests.get(url_origem)
            soup = BeautifulSoup(response.content, 'html.parser')
            links = []
            for link in soup.find_all('a'):
                href = link.get('href')
                if href and href.lower().endswith('.dbc'):
                    links.append(urljoin(url_origem, href))
            print(f"Total de arquivos encontrados: {len(links)}")
            return links
        except Exception as e:
            print(f"Erro ao obter links: {e}")
            return []

    def etapa_3_processar_paralelo(self, lista_links, tabela_destino='beneficiarios_agrupados', max_workers=4):
        """
        Gerencia os workers e grava no banco sequencialmente.
        """
        conn = sqlite3.connect(self.db_path)
        total = len(lista_links)
        processados = 0

        print(f"--- Iniciando Processamento Paralelo ({max_workers} Workers) ---")

        # Inicia o Pool de Processos
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            # Submete todas as tarefas
            # future_to_link é um dicionário para rastrear qual link pertence a qual tarefa
            future_to_link = {executor.submit(processar_arquivo_worker, link): link for link in lista_links}

            for future in as_completed(future_to_link):
                processados += 1
                link = future_to_link[future]
                nome = link.split('/')[-1]

                try:
                    df_resultado = future.result()

                    if df_resultado is not None and not df_resultado.empty:
                        # O momento da escrita no banco é sequencial (Thread Principal)
                        df_resultado.to_sql(tabela_destino, conn, if_exists='append', index=False)
                        print(f"[{processados}/{total}] Salvo: {nome} ({len(df_resultado)} registros)")
                    else:
                        print(f"[{processados}/{total}] Vazio/Ignorado: {nome}")

                except Exception as exc:
                    print(f"[{processados}/{total}] Falha ao recuperar resultado de {nome}: {exc}")

        conn.close()
        print("--- Processo Paralelo Finalizado ---")

# --- EXECUÇÃO ---

if __name__ == "__main__":
    # URL da ANS
    url_ans = "https://dadosabertos.ans.gov.br/FTP/Base_de_dados/Microdados/dados_dbc/beneficiarios/operadoras/"

    # Define quantos núcleos do processador você quer usar
    # Se seu PC for potente, pode aumentar. Geralmente 4 ou 8 é um bom número.
    WORKERS = os.cpu_count() or 4

    bot = ImportadorANSParalelo(db_path='base_ans_paralela.db')

    links = bot.etapa_1_e_2_obter_links(url_ans)

    if links:
        # Executa em paralelo
        bot.etapa_3_processar_paralelo(links, max_workers=WORKERS)

--- Mapeando arquivos em: https://dadosabertos.ans.gov.br/FTP/Base_de_dados/Microdados/dados_dbc/beneficiarios/operadoras/ ---
Total de arquivos encontrados: 58
--- Iniciando Processamento Paralelo (2 Workers) ---
[1/58] Salvo: tb_cc_2011-06.dbc (1413 registros)
[2/58] Salvo: tb_cc_2011-09.dbc (1405 registros)
[3/58] Salvo: tb_cc_2012-03.dbc (1384 registros)
[4/58] Salvo: tb_cc_2011-12.dbc (1391 registros)
[5/58] Salvo: tb_cc_2012-06.dbc (1377 registros)
[6/58] Salvo: tb_cc_2012-09.dbc (1356 registros)
[7/58] Salvo: tb_cc_2012-12.dbc (1346 registros)
[8/58] Salvo: tb_cc_2013-03.dbc (1330 registros)
[9/58] Salvo: tb_cc_2013-06.dbc (1307 registros)
[10/58] Salvo: tb_cc_2013-09.dbc (1295 registros)
[11/58] Salvo: tb_cc_2013-12.dbc (1276 registros)
[12/58] Salvo: tb_cc_2014-03.dbc (1271 registros)
[13/58] Salvo: tb_cc_2014-06.dbc (1260 registros)
[14/58] Salvo: tb_cc_2014-09.dbc (1242 registros)
[15/58] Salvo: tb_cc_2015-03.dbc (1221 registros)
[16/58] Salvo: tb_cc_2014-12.dbc (1243 regist

## Inicio Extração e Tratamento Demonstração Contabeis

In [5]:
def _extrair_info_arquivo_worker(nome_arquivo, url_pasta_ano):
    """Lógica de Regex para descobrir Ano e Trimestre"""
    try:
        nome = nome_arquivo.lower()

        # 1. Extrair ANO da URL da pasta (Mais confiável)
        url_limpa = url_pasta_ano.rstrip('/')
        ano_pasta = url_limpa[-4:]

        if not ano_pasta.isdigit() or len(ano_pasta) != 4:
            match_ano = re.search(r'(20\d{2})', nome)
            if match_ano:
                ano_pasta = match_ano.group(1)
            else:
                return None

        # 2. Extrair TRIMESTRE
        # Regex 1: "1 trimestre", "1_trimestre", "1-trimestre"
        match_longo = re.search(r'([1-4])\s*[-_]?\s*(?:trimestre|tri)', nome)
        if match_longo: return f"{ano_pasta}-T{match_longo.group(1)}"

        # Regex 2: "1t", "4t"
        match_curto = re.search(r'([1-4])t', nome)
        if match_curto: return f"{ano_pasta}-T{match_curto.group(1)}"

        # Regex 3: "t1"
        match_inv = re.search(r't([1-4])', nome)
        if match_inv: return f"{ano_pasta}-T{match_inv.group(1)}"

        return None
    except:
        return None

def processar_zip_worker(args):
    """
    Função que roda em paralelo.
    Recebe uma tupla: (link_do_zip, url_da_pasta_ano)
    Retorna: DataFrame filtrado ou None
    """
    link_zip, url_pasta_ano = args
    nome_arquivo = link_zip.split('/')[-1]

    # Identifica a chave temporal
    chave_trimestre = _extrair_info_arquivo_worker(nome_arquivo, url_pasta_ano)

    if not chave_trimestre:
        return None

    try:
        # 1. Download (timeout aumentado para evitar quedas em arquivos grandes)
        r = requests.get(link_zip, timeout=120)

        # 2. Processamento em Memória
        with zipfile.ZipFile(io.BytesIO(r.content)) as z:
            csvs = [n for n in z.namelist() if n.lower().endswith('.csv')]
            if not csvs: return None

            nome_csv = csvs[0]
            with z.open(nome_csv) as f:
                # Lê tudo como string para não perder zeros a esquerda
                df = pd.read_csv(f, sep=';', encoding='iso-8859-1', dtype=str)

                # Limpa nomes das colunas (Upper + Strip)
                df.columns = [c.upper().strip() for c in df.columns]

                # Verifica se tem a coluna alvo
                if 'CD_CONTA_CONTABIL' in df.columns:
                    # FILTRA CONTA 31
                    df_filtrado = df[df['CD_CONTA_CONTABIL'] == '31'].copy()

                    if not df_filtrado.empty:
                        # Adiciona a chave temporal
                        df_filtrado['ID_TRIMESTRE'] = chave_trimestre

                        # --- TRATAMENTO DE VALOR ---
                        if 'VL_SALDO_FINAL' in df_filtrado.columns:
                            df_filtrado['VL_SALDO_FINAL'] = df_filtrado['VL_SALDO_FINAL'].str.replace('.', '', regex=False)
                            df_filtrado['VL_SALDO_FINAL'] = df_filtrado['VL_SALDO_FINAL'].str.replace(',', '.', regex=False)
                            df_filtrado['VL_SALDO_FINAL'] = pd.to_numeric(df_filtrado['VL_SALDO_FINAL'], errors='coerce')

                        # --- NOVO: SANITIZAÇÃO DE COLUNAS (CORREÇÃO DO ERRO) ---
                        # Aqui definimos EXATAMENTE o que vai para o banco.
                        # Qualquer coluna extra (como VL_SALDO_INICIAL ou DT_CARGA) será ignorada.
                        colunas_finais = ['REG_ANS', 'CD_CONTA_CONTABIL', 'VL_SALDO_FINAL', 'ID_TRIMESTRE']

                        # Verifica quais dessas colunas existem no DF atual (para evitar erro se faltar alguma)
                        colunas_existentes = [c for c in colunas_finais if c in df_filtrado.columns]

                        # Retorna apenas as colunas limpas
                        return df_filtrado[colunas_existentes]

    except Exception as e:
        # Imprime erro mas não para o processo inteiro
        # print(f"   [Erro Worker] Falha em {nome_arquivo}: {e}")
        pass

    return None

# --- CLASSE PRINCIPAL ---

class ExtratorContabilParalelo:
    def __init__(self, db_path='dados_ans.db'):
        self.db_path = db_path
        self.url_base = "https://dadosabertos.ans.gov.br/FTP/PDA/demonstracoes_contabeis/"

    def _mapear_todos_arquivos(self):
        """Varre as pastas de anos e retorna uma lista de tuplas (url_zip, url_ano)"""
        print(f"--- Mapeando estrutura de pastas em: {self.url_base} ---")
        tarefas = []

        try:
            # 1. Pega pastas de Anos
            r = requests.get(self.url_base)
            soup = BeautifulSoup(r.content, 'html.parser')
            links_anos = []
            for link in soup.find_all('a'):
                href = link.get('href')
                if href and re.match(r'\d{4}/', href):
                    links_anos.append(urljoin(self.url_base, href))

            print(f"Anos encontrados: {len(links_anos)}. Buscando ZIPs dentro de cada ano...")

            # 2. Pega ZIPs dentro de cada Ano
            # (Poderíamos paralelizar isso também, mas é rápido o suficiente ser sequencial)
            for url_ano in links_anos:
                try:
                    r_ano = requests.get(url_ano)
                    soup_ano = BeautifulSoup(r_ano.content, 'html.parser')
                    for link in soup_ano.find_all('a'):
                        href = link.get('href')
                        if href and href.lower().endswith('.zip'):
                            full_link = urljoin(url_ano, href)
                            # Guardamos a tupla (Link do Arquivo, Link da Pasta do Ano)
                            tarefas.append((full_link, url_ano))
                except:
                    print(f"Erro ao ler pasta: {url_ano}")

        except Exception as e:
            print(f"Erro no mapeamento: {e}")

        return tarefas

    def executar(self, tabela_destino='demonstracoes_contabeis', max_workers=8):
        # 1. Mapeamento (Sequencial, mas rápido)
        lista_tarefas = self._mapear_todos_arquivos()
        total_arquivos = len(lista_tarefas)

        if total_arquivos == 0:
            print("Nenhum arquivo encontrado.")
            return

        print(f"--- Iniciando Download e Processamento de {total_arquivos} arquivos ---")
        print(f"--- Workers Ativos: {max_workers} ---")

        conn = sqlite3.connect(self.db_path)
        processados = 0
        sucessos = 0

        # 2. Processamento Paralelo
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            # Envia todas as tarefas
            future_to_url = {executor.submit(processar_zip_worker, tarefa): tarefa for tarefa in lista_tarefas}

            for future in as_completed(future_to_url):
                processados += 1
                link, _ = future_to_url[future]
                nome = link.split('/')[-1]

                try:
                    df_resultado = future.result()

                    if df_resultado is not None and not df_resultado.empty:
                        # 3. Escrita no Banco (Sequencial e Segura)
                        df_resultado.to_sql(tabela_destino, conn, if_exists='append', index=False)
                        sucessos += 1
                        trimestre = df_resultado['ID_TRIMESTRE'].iloc[0]
                        print(f"[{processados}/{total_arquivos}] SALVO: {trimestre} ({len(df_resultado)} linhas) -> {nome}")
                    else:
                        print(f"[{processados}/{total_arquivos}] Ignorado/Vazio: {nome}")

                except Exception as exc:
                    print(f"[{processados}/{total_arquivos}] Falha na tarefa {nome}: {exc}")

        conn.close()
        print(f"--- FIM. {sucessos} arquivos processados com sucesso. ---")

# --- EXECUÇÃO ---
if __name__ == "__main__":
    db_nome = 'base_ans_paralela.db'

    # Ajuste o número de workers conforme sua internet e CPU
    # 8 costuma ser um bom número. Se a internet cair, reduza para 4.
    extrator = ExtratorContabilParalelo(db_path=db_nome)
    extrator.executar(max_workers=8)

--- Mapeando estrutura de pastas em: https://dadosabertos.ans.gov.br/FTP/PDA/demonstracoes_contabeis/ ---
Anos encontrados: 19. Buscando ZIPs dentro de cada ano...
--- Iniciando Download e Processamento de 75 arquivos ---
--- Workers Ativos: 8 ---
[1/75] SALVO: 2008-T2 (1299 linhas) -> 2008_2_trimestre.zip
[2/75] SALVO: 2008-T3 (1296 linhas) -> 2008_3_trimestre.zip
[3/75] SALVO: 2008-T1 (1299 linhas) -> 2008_1_trimestre.zip
[4/75] SALVO: 2007-T2 (1366 linhas) -> 2007_2_trimestre.zip
[5/75] SALVO: 2008-T4 (1290 linhas) -> 2008_4_trimestre.zip
[6/75] SALVO: 2007-T1 (1367 linhas) -> 2007_1_trimestre.zip
[7/75] SALVO: 2007-T3 (1368 linhas) -> 2007_3_trimestre.zip
[8/75] SALVO: 2007-T4 (1354 linhas) -> 2007_4_trimestre.zip
[9/75] SALVO: 2009-T1 (1265 linhas) -> 2009_1_trimestre.zip
[10/75] SALVO: 2010-T3 (1256 linhas) -> 2010_3_trimestre.zip
[11/75] SALVO: 2009-T2 (1273 linhas) -> 2009_2_trimestre.zip
[12/75] SALVO: 2010-T1 (1249 linhas) -> 2010_1_trimestre.zip
[13/75] SALVO: 2009-T4 (1278 

## Inicio Extração Dimensão Operadora


## Observação
para realizar a extração da dimensão é necessario rodar esse trecho de codigo na maquina local, pois o google colab contem o IP de fora e essa API so permise realizar requisição em um IP brasileiro

In [ ]:
class ImportadorAPI_ANS:
    def __init__(self, db_path):
        self.db_path = db_path
        self.base_url = "https://dados-abertos-service.pr.ans.gov.br/operadoras"

        # Cria uma sessão que IMPERSONA (finge ser) o Chrome 120
        # Isso resolve o problema de Handshake TLS e bloqueio de bot
        self.session = requests_curl.Session(impersonate="chrome120")

        # Headers adicionais para parecer ainda mais legítimo
        self.session.headers.update({
            'Accept': 'application/json, text/plain, */*',
            'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
            'Referer': 'https://dados-abertos-service.pr.ans.gov.br/',
            'Origin': 'https://dados-abertos-service.pr.ans.gov.br'
        })

    def _requisicao_segura(self, url, params=None):
        """Faz a requisição usando curl_cffi para evitar bloqueios"""
        max_tentativas = 3
        for tentativa in range(max_tentativas):
            try:
                # O curl_cffi não precisa de verify=False geralmente,
                # mas mantemos timeout para não travar
                response = self.session.get(url, params=params, timeout=30)

                if response.status_code == 200:
                    return response.json()
                elif response.status_code == 404:
                    return None
                elif response.status_code in [502, 503, 504]:
                    print(f"   [!] Servidor instável ({response.status_code})... T: {tentativa+1}")
                    time.sleep(3)
                    continue
                else:
                    print(f"   [!] Erro {response.status_code} na URL: {url}")
                    return None

            except Exception as e:
                print(f"   [!] Erro de conexão (Tentativa {tentativa+1}): {e}")
                time.sleep(2)
        return None

    def get_classificacoes(self):
        print("--- Buscando Tabela Auxiliar de Classificações ---")
        url = f"{self.base_url}/classificacoes"
        dados = self._requisicao_segura(url)

        if dados and 'content' in dados:
            df = pd.DataFrame(dados['content'])
            if not df.empty:
                df.rename(columns={'descricao': 'classificacao_descricao_oficial', 'sigla': 'classificacao_sigla'}, inplace=True)
            return df
        return pd.DataFrame()

    def get_operadoras_basico(self):
        print("--- Buscando IDs das Operadoras (Paginação) ---")
        operadoras_lista = []
        pagina = 1
        tem_dados = True

        while tem_dados:
            # Feedback visual a cada página para saber que não travou
            print(f"   Lendo página {pagina}...")

            params = {'page': pagina, 'size': 100}
            dados = self._requisicao_segura(self.base_url, params=params)

            if dados and 'content' in dados and len(dados['content']) > 0:
                items = dados['content']
                for item in items:
                    op = {
                        "registro_ans": item.get("registro_ans"),
                        "url_detalhe": item.get("_links", {}).get("self", {}).get("href")
                    }
                    operadoras_lista.append(op)

                if dados.get('last') is True:
                    tem_dados = False
                else:
                    pagina += 1
            else:
                tem_dados = False

        print(f"   Total de operadoras listadas para consulta: {len(operadoras_lista)}")
        df = pd.DataFrame(operadoras_lista)
        if not df.empty:
            df.drop_duplicates(subset=['registro_ans'], inplace=True)
        return df

    def _buscar_detalhe_operadora(self, row):
        url = row['url_detalhe']
        if not url: return {}

        d = self._requisicao_segura(url)

        if d:
            return {
                "registro_ans": d.get("registro_ans"),
                "cnpj": d.get("cnpj"),
                "razao_social": d.get("razao_social"),
                "nome_fantasia": d.get("nome_fantasia"),
                "ativa": d.get("ativa"),
                "registrada_em": d.get("registrada_em"),
                "descredenciada_em": d.get("descredenciada_em"),
                "descredenciamento_motivo": d.get("descredenciamento_motivo"),
                "classificacao_sigla": d.get("classificacao_sigla"),
                "classificacao_nome": d.get("classificacao_nome"),
                "representante_nome": d.get("representante_nome"),
                "representante_cargo": d.get("representante_cargo"),
                "endereco_logradouro": d.get("endereco_logradouro"),
                "endereco_numero": d.get("endereco_numero"),
                "endereco_complemento": d.get("endereco_complemento"),
                "endereco_bairro": d.get("endereco_bairro"),
                "endereco_cep": d.get("endereco_cep"),
                "endereco_municipio_codigo": d.get("endereco_municipio_codigo"),
                "endereco_municipio_nome": d.get("endereco_municipio_nome"),
                "endereco_uf_sigla": d.get("endereco_uf_sigla"),
                "endereco_valido": d.get("endereco_valido"),
                "telefone_ddd": d.get("telefone_ddd"),
                "telefone_numero": d.get("telefone_numero"),
                "fax_ddd": d.get("fax_ddd"),
                "fax_numero": d.get("fax_numero"),
                "email": d.get("email_comercial") or d.get("email")
            }
        return {}

    def processar(self):
        start_time = time.time()

        # 1. Classificações
        df_classificacoes = self.get_classificacoes()

        # 2. Lista Básica
        df_basico = self.get_operadoras_basico()

        if df_basico.empty:
            print("Nenhuma operadora encontrada ou erro fatal de conexão.")
            return

        # 3. Detalhes em Paralelo
        print("--- Iniciando Extração Detalhada (Multi-Thread) ---")
        detalhes_lista = []
        lista_tarefas = df_basico.to_dict('records')

        # Mantendo 8 workers para não abusar
        with ThreadPoolExecutor(max_workers=8) as executor:
            future_to_op = {executor.submit(self._buscar_detalhe_operadora, item): item for item in lista_tarefas}

            total = len(lista_tarefas)
            for i, future in enumerate(as_completed(future_to_op)):
                if i % 25 == 0:
                    print(f"   Progresso: {i}/{total} operadoras processadas...")

                res = future.result()
                if res:
                    detalhes_lista.append(res)

        df_final = pd.DataFrame(detalhes_lista)

        # 4. Merge Classificações
        if not df_classificacoes.empty and 'classificacao_sigla' in df_final.columns:
            print("--- Validando Classificações ---")
            df_final = pd.merge(df_final, df_classificacoes, on='classificacao_sigla', how='left')

        # 5. Salvar
        print("--- Tratando Dados e Salvando no SQLite ---")
        if df_final.empty:
            print("Erro: Nenhum detalhe foi coletado.")
            return

        df_final = df_final.astype(str)
        df_final.replace(['nan', 'None', 'NAT'], None, inplace=True)

        df_obj = df_final.select_dtypes(['object'])
        df_final[df_obj.columns] = df_obj.apply(lambda x: x.str.strip() if x is not None else x)

        conn = sqlite3.connect(self.db_path)
        df_final.to_sql('dim_operadoras', conn, if_exists='replace', index=False)

        cursor = conn.cursor()
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_dim_op_reg ON dim_operadoras(registro_ans);')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_dim_op_cnpj ON dim_operadoras(cnpj);')
        conn.commit()
        conn.close()

        tempo_total = (time.time() - start_time) / 60
        print(f"--- Processo Concluído em {tempo_total:.2f} minutos. Total registros: {len(df_final)} ---")

if __name__ == "__main__":
    db_nome = 'banco_dim_operadoras.db'
    etl = ImportadorAPI_ANS(db_nome)
    etl.processar()

### O trecho de codigo abaixo serve para juntar os dados da dimensão operadora junto com a base_ans_pararela

In [7]:
class ClonadorComTratamento:
    def __init__(self, db_origem, db_destino):
        self.db_origem = db_origem
        self.db_destino = db_destino
        self.tabela = 'dim_operadoras'

        # 1. Colunas para EXCLUIR
        self.colunas_ignorar = ['_links', 'segmentacoes']

        # 2. Colunas para RENOMEAR (De -> Para)
        self.mapa_renomeacao = {
            'registro_ans': 'registro_operadora',
            'classificacao_nome': 'modalidade',
            'endereco_municipio_nome': 'cidade',
            'endereco_uf_sigla': 'uf',
            'representante_nome': 'representante',
            'representante_cargo': 'cargo_representante',
            'registrada_em': 'data_registro_ans'
        }

    def _obter_colunas_origem(self):
        """Lê o nome das colunas do banco original para montar a query dinâmica"""
        conn = sqlite3.connect(self.db_origem)
        cursor = conn.cursor()
        try:
            # Pega informações da tabela (o segundo item da tupla é o nome da coluna)
            cursor.execute(f"PRAGMA table_info({self.tabela})")
            colunas = [row[1] for row in cursor.fetchall()]
            return colunas
        finally:
            conn.close()

    def executar(self):
        if not os.path.exists(self.db_origem):
            print(f"Erro: Origem {self.db_origem} não encontrada.")
            return

        print(f"--- Iniciando Migração Otimizada com Tratamento ---")
        start_time = time.time()

        # 1. Pega as colunas existentes para montar o SQL
        colunas_originais = self._obter_colunas_origem()
        if not colunas_originais:
            print("Erro: A tabela de origem parece não existir ou está vazia.")
            return

        # 2. Monta a lista de seleção SQL (Select List)
        campos_select = []
        for col in colunas_originais:
            # Regra de Exclusão
            if col in self.colunas_ignorar:
                continue

            # Regra de Renomeação
            if col in self.mapa_renomeacao:
                novo_nome = self.mapa_renomeacao[col]
                campos_select.append(f"{col} AS {novo_nome}")
            else:
                # Mantém o nome original
                campos_select.append(col)

        query_colunas = ", ".join(campos_select)

        # Conexão com Destino
        conn = sqlite3.connect(self.db_destino)
        cursor = conn.cursor()

        try:
            # Configurações de velocidade
            cursor.execute("PRAGMA synchronous = OFF;")
            cursor.execute("PRAGMA journal_mode = MEMORY;")

            print("1. Anexando banco de origem...")
            cursor.execute(f"ATTACH DATABASE '{self.db_origem}' AS origem;")

            print("2. Limpando tabela antiga no destino...")
            cursor.execute(f"DROP TABLE IF EXISTS main.{self.tabela};")

            print("3. Executando cópia transformada (SQL Engine)...")
            # AQUI ESTÁ A MÁGICA: O SQL já faz a seleção e renomeação enquanto copia
            sql_final = f"""
                CREATE TABLE main.{self.tabela} AS
                SELECT {query_colunas}
                FROM origem.{self.tabela};
            """
            cursor.execute(sql_final)

            print("4. Criando índices nas novas colunas...")
            # Atenção: Criamos índices com os NOMES NOVOS
            cursor.execute(f"CREATE INDEX IF NOT EXISTS idx_reg_operadora ON {self.tabela}(registro_operadora);")

            # Verifica se 'cnpj' ainda existe (pois não renomeamos ele, mas ele pode estar na lista)
            if 'cnpj' in colunas_originais and 'cnpj' not in self.colunas_ignorar:
                 cursor.execute(f"CREATE INDEX IF NOT EXISTS idx_cnpj ON {self.tabela}(cnpj);")

            conn.commit()
            print("5. Commit realizado.")

        except Exception as e:
            print(f"Erro crítico: {e}")
            conn.rollback()
        finally:
            try:
                cursor.execute("DETACH DATABASE origem;")
            except:
                pass
            conn.close()

        tempo = time.time() - start_time
        print(f"--- Concluído em {tempo:.4f} segundos ---")

# --- EXECUÇÃO ---
if __name__ == "__main__":
    db_origem = 'banco_dim_operadoras.db'  # Seu banco "sujo" (staging)
    db_destino = 'base_ans_paralela.db' # Seu banco "limpo" (final)

    etl = ClonadorComTratamento(db_origem, db_destino)
    etl.executar()

--- Iniciando Migração Otimizada com Tratamento ---
1. Anexando banco de origem...
2. Limpando tabela antiga no destino...
3. Executando cópia transformada (SQL Engine)...
4. Criando índices nas novas colunas...
5. Commit realizado.
--- Concluído em 0.1039 segundos ---
